In [ ]:
from tools.io import read_json, write_json
from pathlib import Path
import numpy as np
from sklearn.model_selection import KFold

In [ ]:
def print_table(t):
    for ti in t:
        for k,v in ti.items():
            print(f"{k}", end=" ")
            for u, l in v.items():
                print(f"{u},{len(l)}", end=" ")
            print()
        print("="*10)

# K Fold split

In [ ]:
root = Path("../dataset")/"p2"
types = [_ for _ in root.iterdir() if _.is_dir()]
print(types)

In [ ]:
kf = KFold(n_splits=5, random_state=20001122, shuffle=True)

In [ ]:
table = [{ti.parts[-1] : {} for ti in types} for i in range(5)]
table
for ti in types:
    
    imgs = [_ for _ in ti.glob("*.jpg") if 'aug' not in str(_)]
    imgs.sort(key=lambda x:int(x.stem))
    folds = list(kf.split(imgs))

    for fold_number, (train_index, val_index) in enumerate(folds):
        
        train_data = [str(imgs[i]) for i in train_index]
        val_data = [str(imgs[i]) for i in val_index]
        
        table[fold_number][ti.parts[-1]]["train"] = train_data
        table[fold_number][ti.parts[-1]]['valid'] = val_data
    
table

In [ ]:
write_json(table, Path("../dataset")/f"5_folds.json")

# Down sampling

In [ ]:
origin_table = read_json("../dataset/5_folds.json")
aug_table = read_json("../dataset/aug_5_folds.json")

In [ ]:
print_table(origin_table)

In [ ]:
random_idx = np.random.permutation(np.arange(1141))[:470]
for fi in range(len(origin_table)):
    origin_table[fi]["Type0"]["train"] = [ 
        origin_table[fi]["Type0"]["train"][_] for _ in random_idx
    ]
print_table(origin_table)            

In [ ]:
for fi in range(len(origin_table)):
    origin_table[fi]["Type1"] = aug_table[fi]["Type1"]
print_table(origin_table)  

In [ ]:
write_json(origin_table, "../dataset/down_5_folds.json")